In [11]:
# Dependencies 

import pandas as pd
import os
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Specify path of bike data files

directory = 'SourceData'

In [13]:
# Create empty list to store DataFrames

dataframes = []

In [14]:
# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Read the CSV file and append the DataFrame to the list
        df = pd.read_csv(file_path)
        dataframes.append(df)

In [15]:
# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Check data types
print(combined_df.dtypes)


ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object


In [16]:
# Data cleaning

# Convert start and end times to datetime format
combined_df['started_at'] = pd.to_datetime(combined_df['started_at'], format='mixed')
combined_df['ended_at'] = pd.to_datetime(combined_df['ended_at'], format='mixed')

# Sort by start time
combined_df = combined_df.sort_values(by='started_at')
#combined_df['start_city'] = None
#combined_df['start_state'] = None
#combined_df['end_city'] = None
#combined_df['end_state'] = None
combined_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
726626,A51F9BDD9A1D6299,classic_bike,2024-01-01 00:06:21,2024-01-01 00:12:12,Essex Light Rail,JC038,City Hall,JC003,40.712774,-74.036486,40.717732,-74.043845,member
729433,D35D257404F7FC4B,electric_bike,2024-01-01 00:07:37,2024-01-01 00:07:40,Exchange Pl,JC116,Exchange Pl,JC116,40.716466,-74.034401,40.716366,-74.034344,member
687359,38677F2D17A2B5A1,electric_bike,2024-01-01 00:08:26,2024-01-01 00:24:29,Exchange Pl,JC116,Baldwin at Montgomery,JC020,40.716531,-74.034399,40.723659,-74.064194,casual
705727,D25689277AF763BC,classic_bike,2024-01-01 00:08:36,2024-01-01 00:19:23,Hoboken Terminal - River St & Hudson Pl,HB102,12 St & Sinatra Dr N,HB201,40.736068,-74.029127,40.750604,-74.024020,member
708379,2ED1774EBF339BA4,electric_bike,2024-01-01 00:08:44,2024-01-01 00:24:12,Exchange Pl,JC116,Baldwin at Montgomery,JC020,40.716500,-74.034466,40.723659,-74.064194,member


In [18]:
# Get lists of unique stations
start_stations_df = combined_df[['start_station_name', 'start_station_id', 'start_lat', 'start_lng']].drop_duplicates(subset=['start_station_name'])
end_stations_df = combined_df[['end_station_name', 'end_station_id', 'end_lat', 'end_lng']].drop_duplicates(subset=['end_station_name'])


In [19]:
# Use Geoapify reverse coordinates API to find the city and state each station is in

# Define base URL
base_url = 'https://api.geoapify.com/v1/geocode/reverse'

# Define parameters dict and populate with params that will be the same for each request
params = {'type': 'city', 'format': 'json', 'apiKey': geoapify_key}

# Iterate through the hotel_df DataFrame
for index, row in start_stations_df.iterrows():

    # Add lat and long from the row into the parameter dict
    params['lat'] = row['start_lat']
    params['lon'] = row['start_lng']

    # Send GET request to Geoapify and return the start city
    response = requests.get(base_url, params = params).json()
    start_city =  response ["results"][0]["city"]
    start_state = response ["results"][0]["state"]
    
    # Add city to start_city column
    start_stations_df.loc[index, 'start_city'] = start_city
    start_stations_df.loc[index, 'start_state'] = start_state

In [27]:
# Check the data frame
start_stations_df

,start_station_name,start_station_id,start_lat,start_lng,start_city,start_state
726626,Essex Light Rail,JC038,40.712774,-74.036486,Jersey City,New Jersey
729433,Exchange Pl,JC116,40.716466,-74.034401,Jersey City,New Jersey
705727,Hoboken Terminal - River St & Hudson Pl,HB102,40.736068,-74.029127,Hoboken,New Jersey
705566,4 St & River St,HB611,40.740816,-74.027371,Hoboken,New Jersey
714390,Bergen Ave & Sip Ave,JC109,40.731089,-74.064516,Jersey City,New Jersey
...,...,...,...,...,...,...
568171,Liberty St & Nassau St,5105.09,40.731513,-74.025130,Jersey City,New Jersey
575045,W 41 St & 8 Ave,6602.03,40.785404,-74.000300,West New York,New Jersey
570860,Broadway & W 56 St,6847.02,40.757456,-74.020314,Weehawken,New Jersey
604337,North Moore St & Greenwich St,5470.12,40.721097,-74.027794,Jersey City,New Jersey


In [21]:
# Use Geoapify reverse coordinates API to find the city and state each station is in

# Define base URL
base_url = 'https://api.geoapify.com/v1/geocode/reverse'

# Define parameters dict and populate with params that will be the same for each request
params = {'type': 'city', 'format': 'json', 'apiKey': geoapify_key}

# Iterate through the hotel_df DataFrame
for index, row in end_stations_df.iterrows():

    # Add lat and long from the row into the parameter dict
    params['lat'] = row['end_lat']
    params['lon'] = row['end_lng']

    # Send GET request to Geoapify and return the start city
    response = requests.get(base_url, params = params).json()
    end_city =  response ["results"][0]["city"]
    end_state = response ["results"][0]["state"]
    
    # Add city to start_city column
    end_stations_df.loc[index, 'end_city'] = end_city
    end_stations_df.loc[index, 'end_state'] = end_state

In [22]:
# Check the data frame
end_stations_df

,end_station_name,end_station_id,end_lat,end_lng,end_city,end_state
726626,City Hall,JC003,40.717732,-74.043845,Jersey City,New Jersey
729433,Exchange Pl,JC116,40.716366,-74.034344,Jersey City,New Jersey
687359,Baldwin at Montgomery,JC020,40.723659,-74.064194,Jersey City,New Jersey
705727,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,Hoboken,New Jersey
714390,Fairmount Ave,JC093,40.725726,-74.071959,Jersey City,New Jersey
...,...,...,...,...,...,...
182342,7 Ave & Central Park South,6912.01,40.766741,-73.979069,New York,New York
191572,Morton St & Greenwich St,5772.05,40.731150,-74.008870,New York,New York
184541,E 33 St & 5 Ave,6322.01,40.747659,-73.984907,New York,New York
93269,E 67 St & Park Ave,6932.14,40.767801,-73.965921,New York,New York


In [28]:
# Copy data frames and drop everything except ID and city/state
end_stations_df_ids = end_stations_df.drop(columns=['end_station_name', 'end_lat', 'end_lng'])
start_stations_df_ids = start_stations_df.drop(columns=['start_station_name', 'start_lat', 'start_lng'])

In [32]:
combined_df_final = pd.merge(combined_df, end_stations_df_ids, on='end_station_id', how='left')
combined_df_final = pd.merge(combined_df_final, start_stations_df_ids, on='start_station_id', how='left')
combined_df_final

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,end_city,end_state,start_city,start_state
0,A51F9BDD9A1D6299,classic_bike,2024-01-01 00:06:21.000,2024-01-01 00:12:12.000,Essex Light Rail,JC038,City Hall,JC003,40.712774,-74.036486,40.717732,-74.043845,member,Jersey City,New Jersey,Jersey City,New Jersey
1,D35D257404F7FC4B,electric_bike,2024-01-01 00:07:37.000,2024-01-01 00:07:40.000,Exchange Pl,JC116,Exchange Pl,JC116,40.716466,-74.034401,40.716366,-74.034344,member,Jersey City,New Jersey,Jersey City,New Jersey
2,38677F2D17A2B5A1,electric_bike,2024-01-01 00:08:26.000,2024-01-01 00:24:29.000,Exchange Pl,JC116,Baldwin at Montgomery,JC020,40.716531,-74.034399,40.723659,-74.064194,casual,Jersey City,New Jersey,Jersey City,New Jersey
3,D25689277AF763BC,classic_bike,2024-01-01 00:08:36.000,2024-01-01 00:19:23.000,Hoboken Terminal - River St & Hudson Pl,HB102,12 St & Sinatra Dr N,HB201,40.736068,-74.029127,40.750604,-74.024020,member,Hoboken,New Jersey,Hoboken,New Jersey
4,2ED1774EBF339BA4,electric_bike,2024-01-01 00:08:44.000,2024-01-01 00:24:12.000,Exchange Pl,JC116,Baldwin at Montgomery,JC020,40.716500,-74.034466,40.723659,-74.064194,member,Jersey City,New Jersey,Jersey City,New Jersey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912329,B73C8AFF1E71B9CA,electric_bike,2024-10-31 23:47:04.522,2024-10-31 23:52:04.148,Columbus Drive,JC014,JC Medical Center,JC110,40.718355,-74.038914,40.715391,-74.049692,member,Jersey City,New Jersey,Jersey City,New Jersey
912330,EC5CBBB67A22851D,electric_bike,2024-10-31 23:50:22.492,2024-10-31 23:52:47.109,Grand St,JC102,Harborside,JC104,40.715178,-74.037683,40.719252,-74.034234,member,Jersey City,New Jersey,Jersey City,New Jersey
912331,805874AB013327B8,electric_bike,2024-10-31 23:52:38.031,2024-10-31 23:56:39.234,Grove St PATH,JC115,Morris Canal,JC072,40.719410,-74.043090,40.712419,-74.038526,casual,Jersey City,New Jersey,Jersey City,New Jersey
912332,C9F51BFD9D8DE9C1,electric_bike,2024-10-31 23:52:45.119,2024-10-31 23:56:44.237,Hoboken Terminal - River St & Hudson Pl,HB102,8 St & Washington St,HB603,40.736068,-74.029127,40.745984,-74.028199,casual,Hoboken,New Jersey,Hoboken,New Jersey


In [36]:
# Save combined dataframe to file

combined_df_final.to_csv('combined_citibike_nj_data.csv', index=False, header=True)